# Week 3: Basic Document Classification (Part 1)

## Preliminaries 

In [1]:
#necessary library imports and setup introduced previously

import sys
#sys.path.append(r'T:\Departments\Informatics\LanguageEngineering') 
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'\Users\J\Desktop\code\sussex\nlp\labs\resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.tokenize import word_tokenize

from sussex_nltk.corpus_readers import ReutersCorpusReader

Sussex NLTK root directory is \Users\J\Desktop\code\sussex\nlp\labs\resources


## Overview 
In labs this week (and next), the focus will be on the application of sentiment analysis. You will be using a corpus of **book reviews** within an **Amazon review corpus**.

You will be exploring various techniques that can be used to classify the sentiment of Amazon book reviews as either positive or negative. 

You will be developing your own **Word List** and **Naïve Bayes** classifiers and then comparing them to the **NLTK Naïve Bayes** classifier.

## Creating training and testing sets
You will be training and testing various document classifiers. It is essential that the data used in the testing phase is not used during the training phase, since this can lead to overestimating performance. 

We now introduce the `split_data` function (defined in the cell below) which can be used to get separate **training** and **testing** sets.

> Look through the code in the following cell, reading the comments and making sure that you understand each line.

In [2]:
from random import sample # have a look at https://docs.python.org/3/library/random.html to see what random.sample does
from sussex_nltk.corpus_readers import AmazonReviewCorpusReader

 
def split_data(data, ratio=0.7): # when the second argument is not given, it defaults to 0.7
    """
    Given corpus generator and ratio:
     - partitions the corpus into training data and test data, where the proportion in train is ratio,

    :param data: A corpus generator.
    :param ratio: The proportion of training documents (default 0.7)
    :return: a pair (tuple) of lists where the first element of the 
            pair is a list of the training data and the second is a list of the test data.
    """
    
    data = list(data) # data is a generator, so this puts all the generated items in a list
 
    n = len(data)  #Found out number of samples present
    train_indices = sample(range(n), int(n * ratio))          #Randomly select training indices
    test_indices = list(set(range(n)) - set(train_indices))   #Other items are testing indices
 
    train = [data[i] for i in train_indices]           #Use training indices to select data
    test = [data[i] for i in test_indices]             #Use testing indices to select data
 
    return (train, test)                       #Return split data
 

Now we can use this function together with a <code>reader</code> object  to create training and testing data.  Note that the <code>AmazonReviewCorpusReader().category("dvd")</code> returns a reader over just the *dvd* reviews.  The methods <code>positive()</code>, <code>negative()</code> can be called to create readers over reviews classified accordingly to their sentiment.  

In [3]:
#Create an Amazon corpus reader pointing at only dvd reviews
dvd_reader = AmazonReviewCorpusReader().category("dvd")

#The following two lines use the documents function on the Amazon corpus reader. 
#This returns a generator over reviews in the corpus. 
#Each review is an instance of a Python class called AmazonReview. 
#An AmazonReview object contains all the data about a review.
pos_train, pos_test = split_data(dvd_reader.positive().documents())
neg_train, neg_test = split_data(dvd_reader.negative().documents())

#You can also combine the training data
train = pos_train + neg_train


### Exercise 1.1
* Generate 80:20 *training*:*testing* splits of all 4 categories of reviews (*dvd*, *book*, *kitchen* and *electronics*), containing **positive** and **negative** reviews.  
* Record the number of reviews according to category, sentiment and dataset (training or testing) in a Pandas dataframe
* Answer the following questions:
    1. Regarding the *training* data for *books*, how many are a) **positive**, b) **negative**?
    2. Regarding the **negative** *testing* data for, how many are there for each category: a) *dvd*, b) *book*, c) *kitchen* and d) *electronics*? 

In [4]:
trainingRatio = 0.8

def getStats(reader, category):
    pos_train, pos_test = split_data(reader.positive().documents(), trainingRatio)
    neg_train, neg_test = split_data(reader.negative().documents(), trainingRatio)
    return (category, len(pos_train), len(pos_test), len(neg_train), len(neg_test))

book_reader = AmazonReviewCorpusReader().category("book")
kitchen_reader = AmazonReviewCorpusReader().category("kitchen")
electronics_reader = AmazonReviewCorpusReader().category("electronics")

pd.DataFrame([getStats(dvd_reader, "DVD"), getStats(book_reader, "Book"), getStats(kitchen_reader, "Kitchen"), getStats(electronics_reader, "Electronics")], columns=["Category", "Pos train", "Pos test", "Neg train", "Neg test"])

,Category,Pos train,Pos test,Neg train,Neg test
0,DVD,800,200,800,200
1,Book,800,200,800,200
2,Kitchen,800,200,800,200
3,Electronics,800,200,800,200


In [7]:
#1 a) 800 b) 800
#2 a,b,c,d) 200

1

## Creating word lists
The next section will explain how to use a sentiment classifier that bases its decisions on word lists. The classifier requires a list of words indicating positive sentiment, and a second list of words indicating negative sentiment. Given positive and negative word lists, a document's overall sentiment is determined based on counts of occurrences of words that occur in the two lists. In this section we are concerned with the creation of the word lists. We will be considering both hand-crafted lists and automatically generated lists.

### Exercise 2.1

- Create a reasonably long hand-crafted list of words that you think indicate positive sentiment.
- Create a reasonably long hand-crafted list of words that indicate negative sentiment.

Use the following cells to store these lists in the variables `my_positive_word_list` and `my_negative_word_list`.

In [5]:
my_positive_word_list = ["good","great","lovely","amazing","awesome","sweet","fantastic"] # extend this one or put your own list here
my_negative_word_list = ["bad", "terrible", "awful", "dreadful", "horrible", "disgusting"] # extend this one or put your own list here

Next, you should try to derive word lists from the data. One way to do this, is to use the most frequent words in positive reviews as your positive list, and the most frequent words in negative reviews as your negative list. This can be done with the [NLTK <code style="background-color: #F5F5F5;">FreqDist</code>](http://www.nltk.org/api/nltk.html#module-nltk.probability) object. 

> You should make sure you understand the code in the cell below.

In [6]:
from nltk.probability import FreqDist # see http://www.nltk.org/api/nltk.html#module-nltk.probability
from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
from functools import reduce # see https://docs.python.org/3/library/functools.html

#Helper function. Given a list of reviews, return a list of all the words in those reviews
#To understand this look at the description of functools.reduce in https://docs.python.org/3/library/functools.html
def get_all_words(amazon_reviews):
    return reduce(lambda words,review: words + review.words(), amazon_reviews, [])

#A frequency distribution over all words in positive book reviews
pos_freqdist = FreqDist(get_all_words(pos_train))
neg_freqdist = FreqDist(get_all_words(neg_train))

### Exercise 2.2
Explain (in words) how the <code>get_all_words()</code> function works.  Your description should include details about
1. the input
2. the output
3. the algorithm used to generate the output from the input

It takes in a list of amazon review objects and then iterates over them, concatening all the words in each review to a list. Finally it returns said list.

### Exercise 2.3
In the blank code cell below write code that uses the frequency lists, `pos_freqdist` and `neg_freqdist`, created in the above cell and `my_positive_word_list` and `my_negative_word_list` that you manually created earlier to determine whether or not the review data conforms to your expectations. In particular, whether:
- the words you expected to indicate positive sentiment actually occur more frequently in positive reviews than negative reviews
- the words you expected to indicate negative sentiment actually occur more frequently in negative reviews than positive reviews.

Display your findings in a table using pandas.

In [11]:
def getStats(word, expectedSentiment):
    actualSentiment = "Positive" if pos_freqdist[word] > neg_freqdist[word] else "Negative"
    return (word, pos_freqdist[word], neg_freqdist[word], expectedSentiment, actualSentiment)

statsForPosWords = [getStats(word, "Positive") for word in my_positive_word_list]
statsForNegWords = [getStats(word, "Negative") for word in my_negative_word_list]
pd.DataFrame(statsForPosWords + statsForNegWords, columns=["Word", "Pos freq", "Neg freq", "Expected", "Actual"])
    

,Word,Pos freq,Neg freq,Expected,Actual
0,good,265,270,Positive,Negative
1,great,283,140,Positive,Positive
2,lovely,7,4,Positive,Positive
3,amazing,42,13,Positive,Positive
4,awesome,19,8,Positive,Positive
5,sweet,13,4,Positive,Positive
6,fantastic,21,7,Positive,Positive
7,bad,52,194,Negative,Negative
8,terrible,9,40,Negative,Negative
9,awful,2,26,Negative,Negative


### Exercise 2.4
Now, you are going to create positive and negative word lists automatically from the training data. In order to do this:

1. write two new functions to help with automating the process of generating wordlists.

    - `most_frequent_words` - this function should take THREE arguments: 2 frequency distributions and a natural number, k. It should order words by how much more they occur in one frequency distribution than the other.   It should then return the top k highest scoring words. You might want to use the `most_common` method from the `FreqDist` class - this returns a list of word, frequency pairs ordered by frequency.  You might also or alternatively want to use pythons built-in `sorted` function
    - `words_above_threshold` - this function also takes three arguments: 2 frequency distributions and a natural number, k. Again, it should order words by how much more they occur in one distribution than the other.  It should return all of the words that have a score greater than k.

2. Remove punctuation and stopwords from consideration. You can re-use code from near the end of Lab_2_2.
3. Using the training data, create two sets of positive and negative word lists using these functions (1 set with each function). 
4.  Display these 4 lists (possibly in a `Pandas` dataframe?)



In [58]:
def getScoreDict(freqDistA, freqDistB):
    dictA = dict(freqDistA)
    dictB = dict(freqDistB)
    # get words that are in a or b but not both, and their score by finding their count
    scoreDict = {word: freqDistA.get(word, freqDistB.get(word)) for word in freqDistA.keys() ^ freqDistB.keys()}
    # get words that are in a and b, and their score by finding the difference
    scoreDict.update({word: abs(freqDistA[word] - freqDistB[word]) for word in freqDistA.keys() & freqDistB.keys()})
    return scoreDict
    
def most_frequent_words(freqDistA, freqDistB, k):
    scoreDict = getScoreDict(freqDistA, freqDistB)
    sortedPairs = sorted(scoreDict.items(), key=lambda pair: pair[1], reverse=True)
    return [word for word, score in sortedPairs][:k]

def words_above_threshold(freqDistA, freqDistB, k):
    scoreDict = getScoreDict(freqDistA, freqDistB)
    pairsAboveK = filter(lambda pair: pair[1] > k, scoreDict.items())
    return [word for word, score in pairsAboveK]
    
print(most_frequent_words(FreqDist(["hi", "hi", "hi", "my", "my", "a", "c"]), FreqDist(["hi", "my", "a", "a", "a", "a", "b", "b"]), 3))
print(words_above_threshold(FreqDist(["hi", "hi", "hi", "my", "my", "a", "c"]), FreqDist(["hi", "my", "a", "a", "a", "a", "b", "b"]), 2))


    

['a', 'b', 'hi']
['a']


## Creating a word list based classifier
Now you have a number of word lists for use with a classifier. 
> Make sure you understand the following code, which will be used as the basis for creating a word list based classifier.

In [ ]:
from nltk.classify.api import ClassifierI
import random

class SimpleClassifier(ClassifierI): 
    #look at the documentation for ClassifierI https://www.nltk.org/_modules/nltk/classify/api.html
    
    def __init__(self, pos, neg): 
        self._pos = pos 
        self._neg = neg 

    def classify(self, words): 
        score = 0
        
        # add code here that assigns an appropriate value to score
        return "N" if score < 0 else "P"

    def classify_many(self, docs): 
        return [self.classify(doc.words() if hasattr(doc, 'words') else doc) for doc in docs] 

    def labels(self): 
        return ("P", "N")

#Example usage:

classifier = SimpleClassifier(top_pos, top_neg)
classifier.classify("I enjoyed this great movie".split())

### Exercise 3.1

- Copy the above code cell and move it to below this one. Then complete the `classify` method in the above code as specified below.
- Test your classifier on several very simple hand-crafted examples to verify that you have implemented `classify` correctly.

The classifier is initialised with a list of positive words, and a list of negative words. The words of a document are passed to the `classify` method (which is partially completed in the above code fragment). The `classify` method should be defined so that each occurrence of a negative word decrements `score`, and each occurrence of a positive word increments `score`. 
- For `score` less than 0, an "`N`" for negative should be returned.
- For `score` greater than 0,  "`P`" for positive should returned.
- For `score` of 0, the classification decision should be made randomly (see https://docs.python.org/3/library/random.html).


### Exercise 3.2
* Extend your SimpleClassifier class so that it has a `train` function which will derive the wordlists from training data.  You could build a separate class for each way of automatically deriving wordlists (which both inherit from SimpleClassifier) OR a single class which takes an extra parameter at training time.